In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
from google.colab import files
files.upload()

Output hidden; open in https://colab.research.google.com to view.

# ***Data Processing***

In [ ]:
data=pd.read_csv('melb_data.csv')
data=pd.DataFrame(data)
data.head(1)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0


In [ ]:
y=data['Price']
features=['Rooms','Distance','Bathroom','Car','Landsize','BuildingArea','Lattitude','Longtitude']
x=data[features]
trainx,testx,trainy,testy=train_test_split(x,y,random_state=1)

In [ ]:
def model(trainx,trainy,testx,testy):
  model1=RandomForestRegressor(n_estimators=100,random_state=1)
  model1.fit(trainx,trainy)
  p=model1.predict(testx)
  return mean_absolute_error(testy,p)

# ***Dealing With Missing Values***

Worst


In [ ]:
#Case-1    In Case of results:Worst Difficulty:No
colu=[col for col in trainx.columns if trainx[col].isnull().any()]
trainX=trainx.drop(colu,axis=1)
testX=testx.drop(colu,axis=1)
model(trainX,trainy,testX,testy)

175576.61487570422

In [ ]:
#Case-2    In Case of results:Best  Difficulty:Not much
impute=SimpleImputer()
Trainx=pd.DataFrame(impute.fit_transform(trainx))
Testx=pd.DataFrame(impute.transform(testx))
Trainx.columns=trainx.columns
Testx.columns=testx.columns
model(Trainx,trainy,Testx,testy)

166382.04545152767

In [ ]:
#Case-3    In Case of results:Better than 1st worser than 2nd  Difficulty:Sar Dard
copytrainx=trainx.copy()
copytestx=testx.copy()
for col in colu:
  copytrainx[col+'was_missing'] = copytrainx[col].isnull()
  copytestx[col+'was_missing'] = copytestx[col].isnull()
copytrainX=pd.DataFrame(impute.fit_transform(copytrainx))
copytestX=pd.DataFrame(impute.transform(copytestx))
copytrainX.columns=copytrainx.columns
copytestX.columns=copytestx.columns
model(copytrainX,trainy,copytestX,testy)

166930.93238595503

# ***Categorical Variables***

In [ ]:
data.head(1)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0


In [ ]:
Y=data['Price']
features=['Rooms','Distance','Bathroom','Car','Landsize','BuildingArea','Lattitude','Longtitude','Regionname']
X=data[features]
train_X,test_X,train_Y,test_Y=train_test_split(X,Y,random_state=0)

In [ ]:
train_X.head()

,Rooms,Distance,Bathroom,Car,Landsize,BuildingArea,Lattitude,Longtitude,Regionname
664,3,9.2,2.0,2.0,368.0,177.0,-37.78460,145.09350,Southern Metropolitan
3270,2,10.5,1.0,2.0,586.0,80.0,-37.74350,145.04860,Eastern Metropolitan
3873,2,11.2,1.0,1.0,348.0,NaN,-37.86720,145.04320,Southern Metropolitan
13170,3,19.6,1.0,1.0,521.0,NaN,-37.63854,145.05179,Northern Metropolitan
1730,4,11.4,2.0,2.0,687.0,237.0,-37.89310,145.04790,Southern Metropolitan


In [ ]:
s = (train_X.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)
object_cols=['Regionname']

Categorical variables:
['Regionname']


In [ ]:
#Case-1
Train_X=train_X.select_dtypes(exclude='object')
Test_X=test_X.select_dtypes(exclude='object')
model(Train_X,train_Y,Test_X,test_Y)

174257.07483355544

In [ ]:
#Case-2
copytrainx=train_X.copy()
copytestx=test_X.copy()
encoder=OrdinalEncoder()
copytrainx[object_cols]=encoder.fit_transform(train_X[object_cols])
copytestx[object_cols]=encoder.fit_transform(test_X[object_cols])

model(copytrainx,train_Y,copytestx,test_Y)

173440.2490086729

In [ ]:
#Case-3 Version-2
encodedTrainx=pd.get_dummies(train_X,columns=['Regionname'])
encodedTestx=pd.get_dummies(test_X,columns=['Regionname'])
model(encodedTrainx,train_Y,encodedTestx,test_Y)

173954.09222142742